In [ ]:
import subprocess
import os
import tempfile


In [ ]:

# 获取目录下的所有 BAM 文件
bam_dir = "/data/haocheng/data/bam/k562/"
bam_files = [f for f in os.listdir(bam_dir) if f.endswith('.bam')]
bam_files = sorted(bam_files)  # 确保文件按字母顺序排序

# 存储归一化后的结果
result = ""

first_bam = os.path.join(bam_dir, bam_files[0])
second_bam = os.path.join(bam_dir, bam_files[1])

print(f"初次计算覆盖度，使用 BAM 文件: {first_bam} 和 {second_bam}")


temp_file1_path = "/data/haocheng/data/bam/k562/work/coverage1.bed"
temp_file2_path = "/data/haocheng/data/bam/k562/work/coverage2.bed"

print(f"临时文件路径: {temp_file1_path}, {temp_file2_path}")

# 创建一个临时文件用于存储合并后的覆盖度结果
with tempfile.NamedTemporaryFile(delete=False, mode='w', suffix='.bedgraph') as temp_merged_file:
    temp_merged_file_path = temp_merged_file.name
    # 合并覆盖度并直接写入临时文件
    cmd = ["bedtools", "unionbedg", "-i", temp_file1_path, temp_file2_path]
    with open(temp_merged_file_path, 'w') as output:
        subprocess.run(cmd, stdout=output, text=True)

# 使用 awk 处理合并后的覆盖度结果并写入最终文件
final_output_file = "/data/haocheng/data/bed/k562/text_coverage.bedgraph"
awk_command = f"awk '{{print $1\"\\t\"$2\"\\t\"$3\"\\t\"($4 + $5)}}' {temp_merged_file_path} > {final_output_file}"
subprocess.run(awk_command, shell=True)



In [ ]:

# 使用 bedtools unionbedg 进行信号叠加，并将空白地方设置为0
unionbedg_command = f"bedtools unionbedg -i /data/haocheng/data/bed/k562/text_coverage.bedgraph /data/haocheng/sorted_zero_coverage.bed -filler 0 > /data/haocheng/data/bed/k562/text_zero_coverage.bedgraph"
subprocess.run(unionbedg_command, shell=True, check=True)





In [ ]:
output_file = f"/data/haocheng/data/bed/k562/text_chr1_coverage.bedgraph"
# awk 命令
awk_command = f"awk '{{print $1\"\\t\"$2\"\\t\"$3\"\\t\"($4 + $5)}}' {'/data/haocheng/data/bed/k562/text_zero_coverage.bedgraph'} > {output_file}"

# 调用 awk 命令
result =subprocess.run(awk_command, shell=True, check=True)


In [6]:
def compare_bedgraph_files(file1_path, file2_path):
    with open(file1_path, 'r') as file1, open(file2_path, 'r') as file2:
        differences_found = False

        # 逐行比较 file1 和 file2 的行
        for line1 in file1:
            line2 = file2.readline()  # 从 file2 中读取对应的行

            if not line2:  # 如果 file2 已经结束但 file1 还没结束
                differences_found = True
                print(f"文件1 有多余的行:\n文件1: {line1.strip()}\n文件2: 无对应行")
                break

            if line1 != line2:
                differences_found = True
                print(f"不同的行:\n文件1: {line1.strip()}\n文件2: {line2.strip()}")

        # 检查 file2 是否还有剩余行，并仅打印以 chr1 开头的多余行
        for line2 in file2:
            if line2.startswith("chr1\t"):  # 确保只打印 chr1 的行
                differences_found = True
                print(f"文件2 多余的 chr1 行: {line2.strip()}")

        if not differences_found:
            print(f"{file1_path} 和 {file2_path} 的内容完全相同")

file2_path = "/data/haocheng/data/bed/k562/text_chr1_coverage.bedgraph"
file1_path = "/data/haocheng/data/bam/k562/work/output.bedgraph"

compare_bedgraph_files(file1_path, file2_path)


文件2 多余的 chr1 行: chr1	248946437	248956422	0
